# Circuit composition

Copyright (c) 2025 Open Brain Institute

Authors: Michael W. Reimann

last modified: 01.2025

## Summary
This notebook lists the neuronal composition of a (SONATA) circuit model as a Sankey plot.
From the first dropdown menu select the [node set](https://sonata-extension.readthedocs.io/en/latest/sonata_nodeset.html) you want to display the composition of.

From the element in the next cell select at least two properties to display. 

For details, see the [README](README.md).

In [1]:
import bluepysnap as snap
import pandas

from ipywidgets import widgets
import plotly.graph_objects as go

circ_fn = "analysis_circuit/circuit_config.json"
circ = snap.Circuit(circ_fn)

nodeset = widgets.Dropdown(
    options=
    list(circ.node_sets.content.keys()),
    description='Node set')

# Selection of node set

Please select one of the node sets defined in the circuit model from the following menu.

In [2]:
display(nodeset)

Dropdown(description='Node set', options=('All', 'Excitatory', 'Inhibitory', 'L1_DAC', 'L1_HAC', 'L1_LAC', 'L1…

# Selection of properties to display
Please select *between two and eight* properties from the following list of categorical properties defined in the circuit model.

In [3]:
# Get dataframe of all properties and their values
_, val_df = zip(*circ.nodes.get(nodeset.value))
val_df = pandas.concat(val_df, axis=0)

# This type of display only works for categorical properties. In the future, numerical properties could be binned...
is_categorical = val_df.dtypes.apply(lambda _x: isinstance(_x, pandas.CategoricalDtype))
categorical_props = is_categorical[is_categorical].index.values

to_display = widgets.SelectMultiple(options=categorical_props,
                                    index=tuple(range(len(categorical_props)))[:8],
                                    description="Properties") # 8 is the arbitrarily decided maximum

display(to_display)

SelectMultiple(description='Properties', index=(0, 1, 2, 3, 4, 5, 6), options=('etype', 'model_template', 'mod…

In [4]:
# Test of user selection
assert len(to_display.value) >= 2, "Please select AT LEAST 2 properties"
assert len(to_display.value) <= 8, "Please select AT MOST 8 properties"
# Dataframe of only the selected properties
use_df = val_df[list(to_display.value)]

# Create a dataframe for a lookup of every possible (categorical) value of the selected properties to a unique index.
# Index: level 0: Name of the property, level 1: value of the property; values: unique index.
label_idx_lo = pandas.concat([pandas.Series(use_df[col].values.categories.values, name="value")
                              for col in use_df.columns], keys=use_df.columns,
                              names=["column"], axis=0).reset_index(level="column")
label_idx_lo["index"] = range(len(label_idx_lo))
label_idx_lo = label_idx_lo.set_index(["column", "value"])["index"]

# The sankey links are built by iterating over pairs of adjacent columns.
lnk_src = []; lnk_tgt = []; lnk_sz = []

for c1, c2 in zip(use_df.columns[:-1], use_df.columns[1:]):
    # Size of a link: Number of overlapping values.
    counts = use_df[[c1, c2]].value_counts()
    for row_idx, row_val in counts.items():
        lnk_src.append(label_idx_lo[c1][row_idx[0]])
        lnk_tgt.append(label_idx_lo[c2][row_idx[1]])
        lnk_sz.append(row_val)

# Create sankey
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label_idx_lo.index.to_frame()["value"],
      color = "blue"
    ),
    link = dict(
      source = lnk_src, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = lnk_tgt,
      value = lnk_sz
  ))])

fig.update_layout(title_text="Circuit composition", font_size=10)
fig.show()